In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
%matplotlib inline
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
import pandas as pd
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE
import time

In [2]:
# download data
url = 'https://raw.githubusercontent.com/chiphuyen/tf-stanford-tutorials/master/data/'

def maybe_download(filename):
    if not os.path.exists(filename):
        filename, _ = urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    return filename

filename = maybe_download('heart.csv')

In [24]:
raw_data = pd.read_csv('heart.csv')
print(raw_data.columns[:9])
print(raw_data.loc[:,raw_data.columns[:9]])

Index(['sbp', 'tobacco', 'ldl', 'adiposity', 'famhist', 'typea', 'obesity',
       'alcohol', 'age'],
      dtype='object')
     sbp  tobacco    ldl  adiposity  famhist  typea  obesity  alcohol  age
0    160    12.00   5.73      23.11  Present     49    25.30    97.20   52
1    144     0.01   4.41      28.61   Absent     55    28.87     2.06   63
2    118     0.08   3.48      32.28  Present     52    29.14     3.81   46
3    170     7.50   6.41      38.03  Present     51    31.99    24.26   58
4    134    13.60   3.50      27.78  Present     60    25.99    57.34   49
5    132     6.20   6.47      36.21  Present     62    30.77    14.14   45
6    142     4.05   3.38      16.20   Absent     59    20.81     2.62   38
7    114     4.08   4.59      14.60  Present     62    23.11     6.72   58
8    114     0.00   3.83      19.40  Present     49    24.86     2.49   29
9    132     0.00   5.80      30.96  Present     69    30.11     0.00   53
10   206     6.00   2.95      32.27   Absent     72

In [25]:
# Pre-treatment
raw_data = pd.read_csv('heart.csv')

label = raw_data['chd']

data = raw_data.loc[:,raw_data.columns[:9]].replace(['Present', 'Absent'],[0, 1])
data = data.apply(lambda x: (x - np.mean(x)) / (np.max(x) - np.min(x)), axis=0)

all_data = data
all_data['chd'] = label
all_data.describe()
print(all_data)

          sbp   tobacco       ldl  adiposity   famhist     typea   obesity  \
0    0.185241  0.268088  0.068967  -0.064244 -0.584416 -0.063137 -0.023341   
1    0.048489 -0.116207 -0.023019   0.089602  0.415584  0.029171  0.088641   
2   -0.173734 -0.113963 -0.087828   0.192259 -0.584416 -0.016983  0.097111   
3    0.270711  0.123857  0.116354   0.353098 -0.584416 -0.032368  0.186508   
4   -0.036982  0.319370 -0.086434   0.066385 -0.584416  0.106094 -0.001697   
5   -0.054076  0.082191  0.120535   0.302189 -0.584416  0.136863  0.148240   
6    0.031395  0.013280 -0.094796  -0.257531  0.415584  0.090709 -0.164182   
7   -0.207922  0.014242 -0.010476  -0.302286 -0.584416  0.136863 -0.092036   
8   -0.207922 -0.116527 -0.063437  -0.168020 -0.584416 -0.063137 -0.037143   
9   -0.054076 -0.116527  0.073845   0.155336 -0.584416  0.244555  0.127537   
10   0.578403  0.075780 -0.124761   0.191980  0.415584  0.290709  0.024024   
11  -0.036982  0.335396 -0.020929  -0.084384 -0.584416  0.183017

In [4]:
#重组数据集，保证label数量相等
one_label_result = all_data[(all_data.chd == 1)]
zero_label_result = all_data[(all_data.chd == 0)]

one_label_length = len(one_label_result)
zero_label_length = len(zero_label_result)

small_len = one_label_length if one_label_length < zero_label_length else zero_label_length;


one_index = random.sample(list(one_label_result.index.values), small_len)
zero_index = random.sample(list(zero_label_result.index.values), small_len)

new_data = pd.concat([one_label_result.ix[one_index], zero_label_result.ix[zero_index]])
new_data.describe()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


,sbp,tobacco,ldl,adiposity,famhist,typea,obesity,alcohol,age,chd
count,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000,320.000000
mean,0.006581,0.011118,0.018841,0.015173,-0.056291,0.007055,0.005834,-0.000335,0.040298,0.500000
std,0.183460,0.155476,0.151145,0.210646,0.499990,0.156828,0.138021,0.163842,0.282190,0.500783
min,-0.310486,-0.116527,-0.255772,-0.522146,-0.584416,-0.616983,-0.355838,-0.115799,-0.567674,0.000000
25%,-0.122452,-0.107553,-0.083646,-0.124034,-0.584416,-0.082368,-0.087409,-0.115799,-0.179919,0.000000
50%,-0.036982,-0.045373,-0.009082,0.029882,0.415584,0.013786,-0.003893,-0.062704,0.075183,0.500000
75%,0.099771,0.075780,0.091963,0.167644,0.415584,0.106094,0.075781,0.050008,0.274163,1.000000
max,0.680967,0.883473,0.737956,0.477854,0.415584,0.383017,0.644162,0.884201,0.432326,1.000000


In [5]:
# 数据分10份，拿一份做测试集，九份做训练集
train_data_size = int(small_len * 2 * 0.9)
test_data_size = int(small_len * 2 * 0.1)

train_data_index = random.sample(list(new_data.index.values), train_data_size)
train_data = new_data.ix[train_data_index]

test_data_index = list(set(new_data.index.values).difference(set(train_data_index)))
test_data = new_data.ix[test_data_index]

train_label = train_data['chd']
train_data = train_data.loc[:,raw_data.columns[:9]]

test_label = test_data['chd']
test_data = test_data.loc[:,raw_data.columns[:9]]
print(train_data.shape, train_label.shape)
print(test_data.shape, test_label.shape)

(288, 9) (288,)
(32, 9) (32,)


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  


In [10]:
# Define paramaters for the model
learning_rate = 0.01
batch_size = 16
n_epochs = 1000

In [22]:
X = tf.placeholder(dtype = np.float32, shape = [batch_size, 9], name='X')
Y = tf.placeholder(dtype = np.float32, shape = [batch_size, 2], name='Y')

W1 = tf.Variable(tf.random_normal([9, 20]), name='W1')
b1 = tf.Variable(tf.random_normal([1, 20]), name='b1')

W2 = tf.Variable(tf.random_normal([20, 10]), name='W2')
b2 = tf.Variable(tf.random_normal([1, 10]), name='b2')

W3 = tf.Variable(tf.random_normal([10, 2]), name='W3')
b3 = tf.Variable(tf.random_normal([1, 2]), name='b3')

Z1 = tf.matmul(X, W1) + b1
A1 = tf.nn.relu(Z1)
Z2 = tf.matmul(A1, W2) + b2
A2 = tf.nn.relu(Z2)
logits = tf.matmul(A2, W3) + b3

entropy = tf.nn.softmax_cross_entropy_with_logits(labels = Y, logits = logits)

loss = tf.reduce_mean(entropy)

optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

def to_one_hotting(labels):
    return (np.arange(2) == labels[:,None]).astype(np.float32)

with tf.Session() as sess:
    start_time = time.time()
    sess.run(tf.global_variables_initializer())	
    n_batches = int(len(train_data)/batch_size)
    for i in range(n_epochs): 
        total_loss = 0

        for index in range(n_batches):
            X_batch = train_data[index*batch_size:(index+1)*batch_size].values
            Y_batch = train_label[index*batch_size:(index+1)*batch_size].values
            Y_batch = to_one_hotting(Y_batch)
            _, loss_batch, get_entropy, get_logits = sess.run([optimizer, loss, entropy, logits], feed_dict={X: X_batch, Y: Y_batch})
            total_loss += loss_batch
            
        if i%100 == 0:
            print('Average loss epoch :{0}'.format(total_loss/n_batches))

    print('Total time: {0} seconds'.format(time.time() - start_time))

    print('Optimization Finished!')

    # test the model
    n_batches = int(len(test_data)/batch_size)
    total_correct_preds = 0
    for index in range(n_batches):
        X_batch = test_data[index*batch_size:(index+1)*batch_size].values
        Y_batch = test_label[index*batch_size:(index+1)*batch_size].values
        Y_batch = to_one_hotting(Y_batch)
        _, loss_batch, logits_batch = sess.run([optimizer, loss, logits], feed_dict={X: X_batch, Y:Y_batch}) 
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32)) # need numpy.count_nonzero(boolarr) :(
        total_correct_preds += sess.run(accuracy)

    print('Accuracy:',format(total_correct_preds/len(test_data)))

Average loss epoch :1.1135838892724779
Average loss epoch :0.19870074755615658
Average loss epoch :0.11422936318235265
Average loss epoch :0.022384324421485264
Average loss epoch :0.004669598190553693
Average loss epoch :0.001290346626774408
Average loss epoch :0.00040103785431711003
Average loss epoch :0.00013122664789635665
Average loss epoch :4.6420002427617467e-05
Average loss epoch :1.7395373327468e-05
Total time: 14.735323429107666 seconds
Optimization Finished!
Accuracy: 0.53125
